In [1]:
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time
import error_table

from sampling_utils import *
from gen_utils import *
from evaluation_utils import *
from config_db import ConfigDB



In [2]:
num_pics = 10
min_num_cars = 1
max_num_cars = 4

domains = [[52,87],[1,37],[0,1],[0.35,1],[0.5,1.5],[0.25,1],[0.5,1],[0.5,1]]
types = ['int','int','float','float','float','float','float','float']

gen_misclass = True
store_config_db = False
store_err_tab = False

IOMIN_THRESH = 0.4
IOU_THRESH = 0.5
AP_THRESH = 0.75
REC_THRESH = 0.75
NN_PROB_THRESH = 0.5

if gen_misclass:
    checkpoint = '/home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000'
    import squeezedet as nn
    net = nn.init(checkpoint)

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
INFO:tensorflow:Restoring parameters from /home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000


In [ ]:
for j in range(min_num_cars,max_num_cars):
    
    total_attempts = 0
    rejections = 0
    
    # Init data structure
    if store_err_tab:
        err_table = error_table()
    if store_config_db:
        config_db = ConfigDB()
    
    path_data_set = './data/test/'
    #path_data_set = '/home/tommaso/analyzeNN/data/train_0/mis/mis_' + str(j) + '/'
    
    i = 0
    v = 0

    while i < num_pics:

        # Sample configuration
        #num_cars = random.randint(min_num_cars, max_num_cars)
        num_cars = j
        sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
        sample, norm_sample = get_sample(v, 'random', types=types, domains=domains, dims=sample_dims)
        sample[3].sort(reverse=True)
        norm_sample[3].sort(reverse=True)
        
        total_attempts += 1
        
        good_sample = True
        # Check sample distance from stored ones
        if store_config_db:
            sample_pad = pad_sample(sample)
            good_sample = config_db.eps_dist(sample_pad,0.5)
            
            
        if good_sample:
            # Generate image from sample
            img, ground_boxes, img_info = gen_image(sample)

            # Is it a good picture? (cars don't overlap too much)
            save_img = True
            for m in range(len(ground_boxes)):
                for n in range(m+1, len(ground_boxes)):
                    if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                        save_img = False

            # Check misclassification
            if gen_misclass and save_img:
                # Save tmp image    
                tmp_img_file_name = './tmp.png'
                img.save(tmp_img_file_name)

                # Classify img
                (boxes,probs,cats) = nn.classify(tmp_img_file_name, net, NN_PROB_THRESH) 
                print(probs)

                # Not cars
                if any(cats):
                    save_img = True
                else:
                    ap, recall = average_precision(ground_boxes, boxes, IOU_THRESH)
                    save_img =  (ap <= AP_THRESH or recall <= REC_THRESH)
                        
                        

            if save_img:
                
                if store_config_db:
                    config_db.add(sample_pad)            
                
                file_name = 'm_0_' + str(j) + '_' + str(i).zfill(6)
                
                # Save data
                save_image(img, file_name, path_data_set)
                save_label(ground_boxes, file_name, path_data_set)
                
                if store_err_tab:
                    err_table.update_with_elem(img_info)
                    
                i+=1
        else:
            rejections += 1

        if(total_attempts %50 == 0):
            print('gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
    print('END gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
    if store_err_tab:
        err_table_file = open(path_data_set + 'error_table' + str(),'w')
        pickle.dump(err_table,err_table_file)
        err_table_file.close()

[0.8735919]
0.798695298668
[0.91577333]
0.826148179273
[0.87155396]
0.766686884792
[0.90330064]
0.843624890248
[0.77582443]
0.593201208275
[0.84763014]
0.694862002011
[0.89276177]
0.849169689298
[0.84377468]
0.87195000676
[0.86360312]
0.801728822606
[0.85471487]
0.779733372573
[0.89410055]
0.89236481264
[0.92619151]
0.866793687129
[0.85218608]
0.91387797615
[0.90969169]
0.867577101112
[0.87492013]
0.79182026156
[0.86306715]
0.735688400096
[0.91152763]
0.896128458058
[0.89299542]
0.884906810656
[0.90489411]
0.804109392991
[0.8919152]
0.820222828157
[0.87607437]
0.748911438932
[0.85375941]
0.714787318145
[0.89980763]
0.876870052452
[0.78790027]
0.66355121772
[0.78700799]
0.593562579929
[0.90704143]
0.846669255953
[0.86944503]
0.715352817178
[0.90175366]
0.870535460502
[0.89797246]
0.905056537745
[0.88538021]
0.768911161862
[0.86412001]
0.863367992969
[0.85756373]
0.778581108268
[0.88241404]
0.821285488637
[0.90948886]
0.87156616733
[0.87152445]
0.738190391768
[0.88514388]
0.843124429246


[0.82112044]
0.733377425215
[0.85936409]
0.712925352455
[0.89187962]
0.821307336002
[0.86867046]
0.810135360881
[0.84907901]
0.765367822423
[0.89192879]
0.860196105144
[0.9092679]
0.886379438469
[0.87553781]
0.878793697138
[0.78627539]
0.582440231615
[0.9062745]
0.872074006871
[0.82250625]
0.601284603826
[0.79672873]
0.880333835534
[0.8587299]
0.746145482243
[0.85870337]
0.707542631346
[0.8799687]
0.83397316546
[0.89234591]
0.822989137871
gen pics: 300 | misclass pics: 4 | rejections: 0
[0.88326734]
0.800642416547
[0.89182073]
0.901007494276
[0.78080827]
0.57224953565
[0.89904869]
0.841464635918
[0.8965115]
0.83965247427
[0.83909619]
0.740207309781
[0.8811174]
0.850214323803
[0.760028]
0.478785071306
[0.91608971]
0.902754929875
[0.87347329]
0.873412059776
[0.8825438]
0.855680290414
[0.87550575]
0.806374281605
[0.87654591]
0.770225924947
[0.88849336]
0.871978821464
[0.91751313]
0.841628458742
[0.85123187]
0.688238573426
[0.790241]
0.628639031763
[0.85337031]
0.7520252279
[0.89058274]
0.

[0.8779285]
0.762796575213
[0.86151451]
0.750915866074
[0.87753302]
0.821321228307
[0.87520474]
0.810638437965
[0.90210474]
0.818097464962
[0.77432799]
0.701797983611
[0.87083244]
0.787910843145
[0.89694667]
0.858159767435
[0.88488382]
0.804184408511
[0.78511184]
0.647134332622
[0.82948965]
0.711238819047
[0.89191878]
0.889036232974
[0.8857553]
0.836721692357
[0.91805607]
0.818132696067
[0.88330978]
0.803487022761
[0.8799749]
0.848557927672
[0.90725219]
0.838116172573
[0.88599932]
0.847469150121
[0.84555513]
0.705771157514
[0.90077674]
0.864036553644
[0.89753324]
0.867870962407
[0.83399135]
0.812185097074
[0.88349605]
0.753706041989
[0.89047384]
0.883293374035
[0.90407109]
0.853906196635
[0.91219461]
0.832834610099
[0.82157278]
0.681388646018
[0.77812928]
0.674450104527
[0.85883594]
0.679467829589
[0.87037945]
0.801760451347
[0.90072608]
0.809662244848
[0.89238822]
0.895467054289
gen pics: 600 | misclass pics: 6 | rejections: 0
[0.78750074]
0.565143712779
[0.90253073]
0.834990327509
[0

[0.91337204, 0.91088766]
0.121965289294
0.862758928499
0.863192246685
0.114408923145
[0.92240024, 0.85459054]
0.0458256275449
0.912074913734
0.713915219863
0.065567057893
[0.90371519, 0.85369796]
0.0
0.862008183429
0.778948370293
0.0
[0.90448713, 0.90195781]
0.856415940984
0.0488174633594
0.0346201689714
0.889670051093
[0.8964591, 0.88720578]
0.03858997712
0.875124678598
0.871057656202
0.028091396374
[0.87651223, 0.84451461]
0.825470537676
0.131668333689
0.162456672637
0.794000652603
[0.88754863, 0.84266806]
0.027046971891
0.809983961456
0.721707886194
0.0112814896326
[0.88911265, 0.87928617]
0.835078522174
0.0343247106334
0.0251823364494
0.831771942019
gen pics: 200 | misclass pics: 0 | rejections: 0
[0.90485919, 0.88285345]
0.100857467923
0.848405553884
0.815781872773
0.0679515809101
[0.88740277, 0.86199951]
0.768268246766
0.157685499838
0.200884342386
0.838202308531
[0.9018206, 0.8554703]
0.123917935964
0.904781322698
0.842908635452
0.127999636219
[0.87363273, 0.85306257]
0.82071045

[0.88621253, 0.85711765]
0.00405394668826
0.776803711999
0.773047759476
0.0255946281093
[0.87256032, 0.85403848]
0.0
0.798356317234
0.718722596371
0.0
[0.850573, 0.80234241]
0.0173033143017
0.771858541894
0.655822960222
0.0
gen pics: 750 | misclass pics: 3 | rejections: 0
[0.91100097, 0.850869]
0.0302444396945
0.845604388606
0.705629324724
0.00119632750835
[0.9197492, 0.8996945]
0.844842335841
0.0491602856216
0.0411860117718
0.857553587901
[0.90673214, 0.85656834]
0.0956663633894
0.869486200034
0.892872726605
0.106455767572
[0.84782046, 0.82206756]
0.72445232412
0.0300332756998
0.0426777687128
0.929855318411
[0.8740918, 0.8439914]
0.786895202986
0.0593498738753
0.0585706686704
0.766905428523
[0.89996248, 0.85093814]
0.0
0.814086332674
0.721820493123
0.0
[0.86239254, 0.84280849]
0.770646829502
0.0201489303035
0.0
0.816042717886
[0.89402676, 0.86727905]
0.0
0.876716702496
0.813137147001
0.0
[0.85549015, 0.85042644]
0.735612555039
0.11935510141
0.162604120467
0.814762621756
[0.90280718, 0

[0.90642458, 0.80215043, 0.62768114]
0.0975839819438
0.832061190467
0.7265551279
0.0814793831431
0.24632727708
0.0
[0.91509914, 0.90213126]
0.142092432422
0.846717503175
0.892769516812
0.14283869059
[0.89058626, 0.84477156, 0.79254204]
0.0
0.830465761875
0.715889229843
0.0
0.187792823684
0.148341954441
END gen pics: 1257 | misclass pics: 10 | rejections: 0
gen pics: 50 | misclass pics: 0 | rejections: 0
gen pics: 100 | misclass pics: 0 | rejections: 0
[0.91005182, 0.88804477, 0.82913166]
0.0
0.0851594468733
0.874202060837
0.85676286558
0.00512556187722
0.0
0.0237336810811
0.839082545092
0.101084421419
gen pics: 150 | misclass pics: 0 | rejections: 0
gen pics: 200 | misclass pics: 0 | rejections: 0
gen pics: 250 | misclass pics: 0 | rejections: 0
gen pics: 300 | misclass pics: 0 | rejections: 0
gen pics: 350 | misclass pics: 0 | rejections: 0
gen pics: 400 | misclass pics: 0 | rejections: 0
[0.91436511, 0.90125591, 0.84761828]
0.131168170553
0.879269422505
0.155719434079
0.840411510344


[0.89787847, 0.88130981, 0.87678701]
0.844686900003
0.0
0.0890248491948
0.0988724159876
0.0
0.829935548976
0.0
0.83261607204
0.0
gen pics: 3700 | misclass pics: 1 | rejections: 0
gen pics: 3750 | misclass pics: 1 | rejections: 0
[0.91285294, 0.89013106, 0.8694641]
0.0
0.875712329871
0.177637801588
0.0
0.123928245058
0.868429227132
0.78497990916
0.0
0.0
[0.92518014, 0.9090094, 0.9089613]
0.135810255708
0.859537996508
0.219298199913
0.820928328349
0.167040109559
0.0
0.0
0.164154444315
0.855959287386
gen pics: 3800 | misclass pics: 1 | rejections: 0
gen pics: 3850 | misclass pics: 1 | rejections: 0
gen pics: 3900 | misclass pics: 1 | rejections: 0
[0.90059441, 0.89204013, 0.87021279]
0.0
0.0
0.837252812577
0.169628947444
0.848795154691
0.0
0.773833686893
0.13881491868
0.0
gen pics: 3950 | misclass pics: 1 | rejections: 0
gen pics: 4000 | misclass pics: 1 | rejections: 0
gen pics: 4050 | misclass pics: 1 | rejections: 0
gen pics: 4100 | misclass pics: 1 | rejections: 0
gen pics: 4150 | mis

[0.89688683, 0.89444274, 0.86491024]
0.0380679454628
0.0
0.837393784565
0.0171822171513
0.82831409104
0.0
0.845173975274
0.0
0.0799514872578
gen pics: 7500 | misclass pics: 3 | rejections: 0
[0.91331291, 0.88657182, 0.86064082]
0.0456836025621
0.87724657865
0.0
0.767768588893
0.0391072592919
0.0
0.0
0.0
0.89006069079
gen pics: 7550 | misclass pics: 3 | rejections: 0
gen pics: 7600 | misclass pics: 3 | rejections: 0
gen pics: 7650 | misclass pics: 3 | rejections: 0
[0.91367966, 0.89885581, 0.85890067]
0.0
0.0
0.863566175025
0.0
0.874017678277
0.0
0.864103581244
0.0
0.0
gen pics: 7700 | misclass pics: 3 | rejections: 0
gen pics: 7750 | misclass pics: 3 | rejections: 0
gen pics: 7800 | misclass pics: 3 | rejections: 0
[0.90923834, 0.86391109, 0.79641736]
0.178641826856
0.0
0.888745534076
0.882123373753
0.141756720302
0.199344262911
0.17425068444
0.753724493355
0.0
gen pics: 7850 | misclass pics: 3 | rejections: 0
gen pics: 7900 | misclass pics: 3 | rejections: 0
gen pics: 7950 | misclass 

gen pics: 11350 | misclass pics: 3 | rejections: 0
[0.88235098, 0.86995012, 0.8029294]
0.0
0.805377337395
0.0
0.728004800163
0.0
0.0826065844279
0.0680197973524
0.0
0.856842798707
gen pics: 11400 | misclass pics: 3 | rejections: 0
gen pics: 11450 | misclass pics: 3 | rejections: 0
gen pics: 11500 | misclass pics: 3 | rejections: 0
gen pics: 11550 | misclass pics: 3 | rejections: 0
gen pics: 11600 | misclass pics: 3 | rejections: 0
gen pics: 11650 | misclass pics: 3 | rejections: 0
gen pics: 11700 | misclass pics: 3 | rejections: 0
[0.90857196, 0.90828186, 0.89798462]
0.0
0.0988635371116
0.922521699941
0.826577228402
0.24139830799
0.0
0.17786861551
0.855232913137
0.110714247471
[0.89071035, 0.87314451, 0.83968914]
0.0948082416018
0.0
0.874168806005
0.764238819081
0.163341911486
0.113298430442
0.20482052216
0.768146398445
0.0
gen pics: 11750 | misclass pics: 3 | rejections: 0
[0.89345133, 0.79382354, 0.7864278, 0.7595892]
0.0247997133691
0.0
0.86311776905
0.0289058936789
0.656999679445
0

gen pics: 14850 | misclass pics: 6 | rejections: 0
gen pics: 14900 | misclass pics: 6 | rejections: 0
gen pics: 14950 | misclass pics: 6 | rejections: 0
[0.88929707, 0.86996359, 0.84998292]
0.0712225924523
0.0
0.873147767329
0.0338439859281
0.746531607349
0.0
0.7634640663
0.0516188806309
0.0718450246131
gen pics: 15000 | misclass pics: 6 | rejections: 0
gen pics: 15050 | misclass pics: 6 | rejections: 0
gen pics: 15100 | misclass pics: 6 | rejections: 0
gen pics: 15150 | misclass pics: 6 | rejections: 0
gen pics: 15200 | misclass pics: 6 | rejections: 0
gen pics: 15250 | misclass pics: 6 | rejections: 0
[0.79036373, 0.77801901, 0.73456806, 0.71505862]
0.0
0.588148209024
0.0641437993877
0.58762526153
0.0
0.148006832965
0.337667201905
0.0
0.485396858761
0.0
0.219165476618
0.289942913601
gen pics: 15300 | misclass pics: 7 | rejections: 0
gen pics: 15350 | misclass pics: 7 | rejections: 0
gen pics: 15400 | misclass pics: 7 | rejections: 0
gen pics: 15450 | misclass pics: 7 | rejections: 0
